<a href="https://colab.research.google.com/github/leshaz44/Aliaksei/blob/main/Task_3/Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import sqlite3

# Загрузка файла CSV
df = pd.read_csv('data_Salary2.csv')
# Проверка загрузки
try:
    df = pd.read_csv('data_Salary2.csv')  # Читаем CSV-файл

    if len(df) > 0:               # Проверяем наличие записей в таблице
        print("Файл CSV успешно загружен.")
    else:
        print("Файл пустой!")
except FileNotFoundError:
    print("Ошибка: Файл data_Salary2.csv не найден.")
except Exception as e:
    print(f"Произошла ошибка при чтении файла: {e}")


# Подключение к базе данных SQLite
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# Создание таблицы
cursor.execute('''CREATE TABLE IF NOT EXISTS table_name (
  id INTEGER PRIMARY KEY,
  Age INTEGER TEXT,
  Gender TEXT,
  Education TEXT,
  Job TEXT,
  Experience FLOAT,
  Salary FLOAT)''')

# Импортируем данные из DataFrame в созданную таблицу
df.to_sql('table_name', con=conn, index=False, if_exists='append')

# Выбор определенных столбцов и фильтрация результатов
cursor.execute("SELECT Salary, Age FROM table_name WHERE age > 30")
result = cursor.fetchall()
for row in result:
    print(row)

# Закрытие соединения
conn.commit()
conn.close()

Выходные данные были обрезаны до нескольких последних строк (5000).
(150000.0, 44)
(70000.0, 33)
(90000.0, 35)
(170000.0, 43)
(50000.0, 31)
(150000.0, 41)
(160000.0, 44)
(60000.0, 33)
(85000.0, 35)
(180000.0, 45)
(110000.0, 36)
(160000.0, 44)
(55000.0, 31)
(110000.0, 38)
(180000.0, 48)
(50000.0, 31)
(130000.0, 40)
(160000.0, 45)
(60000.0, 33)
(95000.0, 36)
(170000.0, 47)
(90000.0, 34)
(150000.0, 44)
(70000.0, 33)
(90000.0, 35)
(170000.0, 43)
(50000.0, 31)
(150000.0, 41)
(160000.0, 44)
(60000.0, 33)
(85000.0, 35)
(170000.0, 43)
(90000.0, 34)
(150000.0, 44)
(160000.0, 31)
(170000.0, 32)
(190000.0, 33)
(155000.0, 31)
(185000.0, 34)
(190000.0, 33)
(155000.0, 31)
(170000.0, 32)
(190000.0, 33)
(155000.0, 31)
(185000.0, 34)
(190000.0, 33)
(155000.0, 31)
(170000.0, 32)
(190000.0, 33)
(155000.0, 31)
(185000.0, 34)
(190000.0, 33)
(155000.0, 31)
(170000.0, 32)
(190000.0, 33)
(155000.0, 31)
(185000.0, 34)
(190000.0, 33)
(155000.0, 31)
(185000.0, 34)
(190000.0, 33)
(155000.0, 31)
(170000.0, 32)
(19